In [17]:
import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format
pd.options.display.max_rows = 100
pd.options.display.max_columns = 40
import numpy as np
import os,random, math
from tqdm import tqdm
from copy import deepcopy
from collections import Counter

# Visualization
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import font_manager, rc
plt.rcParams['font.family'] = 'NanumGothic'
import platform
if platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
    rc('font', family='AppleGothic')

matplotlib.rcParams['axes.unicode_minus'] = False

# Warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

import datetime
import re

from dateutil.relativedelta import relativedelta
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error as mae

import folium
from folium import plugins
import json
import geopandas as gpd
from geopandas import gpd
from haversine import haversine
from shapely.geometry import Point, shape, mapping,Polygon
from shapely.ops import unary_union

In [18]:
censor = gpd.read_file('/Users/seungji/Library/Mobile Documents/com~apple~CloudDocs/STDA/Project/data/censor.shp')

In [19]:
censor_g = censor[['serial','geometry']].drop_duplicates().reset_index(drop=True)

In [20]:
censor.head()

,serial,date,fp,gu,juso,lat,lon,subway,bus,building_c,building_a,pop,geometry
0,"2,992.0000",2021-01-04,"6,594.7167",???,????? ??? ??? 733-28,37.4895,126.8848,"15,223.5000","1,248.3000",22,"11,335.4700","4,508.0000",POINT (37.489 126.885)
1,"2,992.0000",2021-01-05,"6,290.3333",???,????? ??? ??? 733-28,37.4895,126.8848,"14,932.0000","1,239.3000",22,"11,335.4700","4,508.0000",POINT (37.489 126.885)
2,"2,992.0000",2021-01-06,"6,222.3667",???,????? ??? ??? 733-28,37.4895,126.8848,"15,729.0000","1,254.0000",22,"11,335.4700","4,508.0000",POINT (37.489 126.885)
3,"2,992.0000",2021-01-07,"5,919.0333",???,????? ??? ??? 733-28,37.4895,126.8848,"16,100.5000","1,319.9000",22,"11,335.4700","4,508.0000",POINT (37.489 126.885)
4,"2,992.0000",2021-01-08,"5,982.3000",???,????? ??? ??? 733-28,37.4895,126.8848,"14,750.0000","1,207.8000",22,"11,335.4700","4,508.0000",POINT (37.489 126.885)


In [21]:
censor['month'] = list(map(lambda x: x[5:7], censor.date))

In [23]:
censor.shape

(26319, 14)

In [24]:
censor.index = pd.to_datetime(censor.date)

In [25]:
censor['week'] = censor.date.to_period('W').index

In [26]:
sensor_wk = censor.groupby(['serial','week']).mean().reset_index()

In [27]:
sensor_wk

,serial,week,fp,lat,lon,subway,bus,building_c,building_a,pop
0,"2,992.0000",2021-01-04/2021-01-10,"6,071.1738",37.4895,126.8848,"12,525.2143","1,106.3429",22.0000,"11,335.4700","4,508.0000"
1,"2,992.0000",2021-01-11/2021-01-17,"6,688.2619",37.4895,126.8848,"12,799.2857","1,142.4000",22.0000,"11,335.4700","4,508.0000"
2,"2,992.0000",2021-01-18/2021-01-24,"6,170.6548",37.4895,126.8848,"12,993.1429","1,164.8286",22.0000,"11,335.4700","4,508.0000"
3,"2,992.0000",2021-01-25/2021-01-31,"4,498.7917",37.4895,126.8848,"14,442.4167","1,249.3333",22.0000,"11,335.4700","4,508.0000"
4,"2,992.0000",2021-02-01/2021-02-07,"5,003.6472",37.4895,126.8848,"13,030.0833","1,208.8333",22.0000,"11,335.4700","4,508.0000"
...,...,...,...,...,...,...,...,...,...,...
3901,"4,050.0000",2021-09-20/2021-09-26,70.0952,37.5729,126.9556,"4,555.0714",217.4821,11.0000,"67,916.9400","3,152.0000"
3902,"4,050.0000",2021-09-27/2021-10-03,61.5833,37.5729,126.9556,"6,951.7500",352.6562,11.0000,"67,916.9400","3,152.0000"
3903,"4,050.0000",2021-10-04/2021-10-10,63.6222,37.5729,126.9556,"5,970.8333",303.4792,11.0000,"67,916.9400","3,152.0000"
3904,"4,050.0000",2021-10-11/2021-10-17,54.3762,37.5729,126.9556,"6,242.7857",298.8482,11.0000,"67,916.9400","3,152.0000"


In [28]:
sensor_wk['week2'] = list(map(lambda x: str(x)[:10], sensor_wk.week))

In [29]:
sensor_wk.week = sensor_wk.week.astype(str)

In [31]:
sensor_wk = pd.merge(sensor_wk, censor_g, on = 'serial',how = 'left')

In [32]:
sensor_wk.to_file('/Users/seungji/Library/Mobile Documents/com~apple~CloudDocs/STDA/Project/data/sensor_wk.shp')

In [124]:
censor = gpd.read_file('/Users/seungji/Library/Mobile Documents/com~apple~CloudDocs/STDA/Project/data/censor.shp')

In [125]:
censor_g = censor[['serial','geometry']].drop_duplicates().reset_index(drop=True)

In [126]:
censor.head()

,serial,date,fp,gu,juso,lat,lon,subway,bus,building_c,building_a,pop,geometry
0,"2,992.0000",2021-01-04,"6,594.7167",???,????? ??? ??? 733-28,37.4895,126.8848,"15,223.5000","1,248.3000",22,"11,335.4700","4,508.0000",POINT (37.489 126.885)
1,"2,992.0000",2021-01-05,"6,290.3333",???,????? ??? ??? 733-28,37.4895,126.8848,"14,932.0000","1,239.3000",22,"11,335.4700","4,508.0000",POINT (37.489 126.885)
2,"2,992.0000",2021-01-06,"6,222.3667",???,????? ??? ??? 733-28,37.4895,126.8848,"15,729.0000","1,254.0000",22,"11,335.4700","4,508.0000",POINT (37.489 126.885)
3,"2,992.0000",2021-01-07,"5,919.0333",???,????? ??? ??? 733-28,37.4895,126.8848,"16,100.5000","1,319.9000",22,"11,335.4700","4,508.0000",POINT (37.489 126.885)
4,"2,992.0000",2021-01-08,"5,982.3000",???,????? ??? ??? 733-28,37.4895,126.8848,"14,750.0000","1,207.8000",22,"11,335.4700","4,508.0000",POINT (37.489 126.885)


In [129]:
sensor_mt = censor.groupby(['serial','month']).mean().reset_index()

In [134]:
sensor_mt

,serial,month,fp,lat,lon,subway,bus,building_c,building_a,pop,geometry
0,"2,992.0000",2021-01-01,"5,907.5327",37.4895,126.8848,"13,143.6296","1,162.6296",22.0000,"11,335.4700","4,508.0000",POINT (37.489 126.885)
1,"2,992.0000",2021-02-01,"5,428.4716",37.4895,126.8848,"12,508.0741","1,171.7444",22.0000,"11,335.4700","4,508.0000",POINT (37.489 126.885)
2,"2,992.0000",2021-03-01,"6,029.9231",37.4895,126.8848,"13,881.1452","1,263.1581",22.0000,"11,335.4700","4,508.0000",POINT (37.489 126.885)
3,"2,992.0000",2021-04-01,"4,529.7779",37.4895,126.8848,"14,318.8000","1,284.7633",22.0000,"11,335.4700","4,508.0000",POINT (37.489 126.885)
4,"2,992.0000",2021-05-01,"5,002.5532",37.4895,126.8848,"13,245.8710","1,255.9258",22.0000,"11,335.4700","4,508.0000",POINT (37.489 126.885)
...,...,...,...,...,...,...,...,...,...,...,...
925,"4,050.0000",2021-06-01,94.4656,37.5729,126.9556,"6,572.8500",322.2021,11.0000,"67,916.9400","3,152.0000",POINT (37.573 126.956)
926,"4,050.0000",2021-07-01,75.4935,37.5729,126.9556,"5,468.6452",289.0786,11.0000,"67,916.9400","3,152.0000",POINT (37.573 126.956)
927,"4,050.0000",2021-08-01,68.2360,37.5729,126.9556,"5,361.0000",280.2056,11.0000,"67,916.9400","3,152.0000",POINT (37.573 126.956)
928,"4,050.0000",2021-09-01,83.1817,37.5729,126.9556,"5,900.3846",293.3894,11.0000,"67,916.9400","3,152.0000",POINT (37.573 126.956)


In [131]:
censor['month'] = list(map(lambda x: x[:7]+'-01', censor.date))

In [132]:
sensor_mt.month = sensor_mt.month.astype('str')

In [133]:
sensor_mt = pd.merge(sensor_mt, censor_g, on = 'serial',how = 'left')

In [135]:
sensor_mt.to_file('/Users/seungji/Library/Mobile Documents/com~apple~CloudDocs/STDA/Project/data/sensor_mt.shp')

In [10]:
censor_oct = censor[censor.month == '10']
censor_oct = censor_oct.groupby('serial').mean().reset_index()

In [14]:
censor_g = censor[['serial','geometry']].drop_duplicates().reset_index(drop=True)

In [16]:
censor_oct = pd.merge(censor_oct, censor_g, on = 'serial',how = 'left')

In [19]:
censor_oct.to_file('/Users/seungji/Library/Mobile Documents/com~apple~CloudDocs/STDA/Project/data/censor_oct.shp')

In [23]:
censor_oct

,serial,fp,lat,lon,subway,bus,building_c,building_a,pop,geometry
0,"2,992.0000","4,300.2149",37.4895,126.8848,"13,405.8421","1,289.4632",22.0000,"11,335.4700","4,508.0000",POINT (37.489 126.885)
1,"2,993.0000","3,851.4956",37.6374,127.0390,"7,283.8750",586.5902,21.0000,"6,650.0300","4,896.0000",POINT (37.637 127.039)
2,"2,994.0000","5,552.8974",37.4882,126.8851,"13,405.8421","1,114.5263",21.0000,"5,084.7400","5,513.0000",POINT (37.488 126.885)
3,"2,995.0000",NaN,37.5025,126.8510,"2,546.0987",373.6684,19.0000,"5,717.5500","3,451.0000",POINT (37.502 126.851)
4,"2,996.0000",NaN,37.5017,126.8508,"5,092.1974",277.2538,20.0000,"7,423.8000","4,199.0000",POINT (37.502 126.851)
5,"2,997.0000","4,705.9482",37.5344,126.8380,"12,248.7237",882.3421,20.0000,"17,808.0200","6,790.0000",POINT (37.534 126.838)
6,"2,998.0000","4,828.1167",37.5352,126.8399,"12,248.7237",721.0632,22.0000,"11,136.1700","7,312.0000",POINT (37.535 126.840)
7,"2,999.0000","2,922.8061",37.4973,127.0513,"12,598.0526",963.6541,22.0000,"105,169.6600","7,053.0000",POINT (37.497 127.051)
8,"3,000.0000","4,434.0728",37.5087,127.0228,"21,518.3947","1,275.2566",20.0000,"25,008.2700","3,531.0000",POINT (37.509 127.023)
9,"3,001.0000","2,632.8930",37.4890,127.0679,"3,371.1053",265.3450,14.0000,"95,036.8800","2,638.0000",POINT (37.489 127.068)


In [6]:
censor_july = censor[censor.month == '07']
censor_july = censor_july.groupby('serial').mean().reset_index()

In [7]:
censor_g = censor[['serial','geometry']].drop_duplicates().reset_index(drop=True)

In [8]:
censor_july = pd.merge(censor_july, censor_g, on = 'serial',how = 'left')

In [9]:
censor_july.to_file('/Users/seungji/Library/Mobile Documents/com~apple~CloudDocs/STDA/Project/data/censor_july.shp')

In [14]:
censor_may = censor[censor.month == '05']
censor_may = censor_may.groupby('serial').mean().reset_index()

In [15]:
censor_g = censor[['serial','geometry']].drop_duplicates().reset_index(drop=True)

In [16]:

censor_may = pd.merge(censor_may, censor_g, on = 'serial',how = 'left')

In [17]:
censor_may.to_file('/Users/seungji/Library/Mobile Documents/com~apple~CloudDocs/STDA/Project/data/censor_may.shp')

In [18]:
censor_march = censor[censor.month == '03']
censor_march = censor_march.groupby('serial').mean().reset_index()

In [19]:
censor_g = censor[['serial','geometry']].drop_duplicates().reset_index(drop=True)

In [20]:

censor_march = pd.merge(censor_march, censor_g, on = 'serial',how = 'left')

In [21]:
censor_march.to_file('/Users/seungji/Library/Mobile Documents/com~apple~CloudDocs/STDA/Project/data/censor_march.shp')